In [3]:
import EDS as eds

required_para = eds.required_parameters(    
    spray_number = [1, 2, 3],
    spray_moment = [30, 45, 60],
    spray_eff = [0.5, 0.5, 0.5],
    
    # for ["Susceptible", "Moderate", "Resistant"]
    p_opt = [7, 10, 14],
    rc_opt_par = [0.35, 0.25, 0.15],
    rrlex_par = [0.1, 0.01, .0001]
)

result = eds.calculate_disease_severity(    
    data_path = "Original_Code/Data.csv",
    info_path = "Original_Code/Info.csv",
    number_of_repeat_year = 1,    
    crop_mechanistic = ["Corn", "Soy"],
    crop_parameters_path = ["Original_Code/Parameters_Corn.xlsx", "Original_Code/Parameters_Soy.xlsx"],    
    number_applications_list = [0],
    genetic_mechanistic_list = ["Susceptible", "Moderate", "Resistant"],    
    required_para = required_para,    
    path_result = None
)

result.to_csv(f"results.csv", index=None)

/home/pooya/miniconda3/lib/python3.9/site-packages/EDS/location_data.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["info_id"] = row[1]["ID"] + "_" + str(row[1]["year"]) + "_" + str(row[1]["planting_date"]) + "_" + str(row[1]["obs_planting_delta"]) + "_" + row[1]["Crop"]
/home/pooya/miniconda3/lib/python3.9/site-packages/EDS/location_data.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["year"] = row[1]["year"]
/home/pooya/miniconda3/lib/python3.9/site-packages/EDS/location_data.py:44: Setti

# Step to Step Run

In [ ]:
import datetime
import itertools
import math
import numpy as np
import pandas as pd

from Create_Package_Original.EDS.required_parameters import *
from Create_Package_Original.EDS.calculate_disease_severity import *



required_para = required_parameters(    
    spray_number = [1, 2, 3],
    spray_moment = [30, 45, 60],
    spray_eff = [0.5, 0.5, 0.5],
    
    # for ["Susceptible", "Moderate", "Resistant"]
    p_opt = [7, 10, 14],
    rc_opt_par = [0.35, 0.25, 0.15],
    rrlex_par = [0.1, 0.01, .0001]
)


result = calculate_disease_severity(    
    data_path = "Original_Code/data/Data.csv",
    info_path = "Original_Code/data/Info.csv",
    add_year = True,
    
    crop_mechanistic = ["Corn", "Soy"],
    crop_parameters_path = ["Original_Code/data/Parameters_Corn.xlsx", "Original_Code/data/Parameters_Soy.xlsx"],
    
    number_applications_list = [0, 1, 2, 3],
    genetic_mechanistic_list = ["Susceptible", "Moderate", "Resistant"],
    
    required_para = required_para,
    
    path_result = None
)


result.to_csv(f"results.csv", index=None)

In [1]:
import pandas as pd

In [8]:
data = pd.read_csv("Data.csv")

for id, df in data.groupby("ID"):
    print(df["maximum_temperature"][(df["maximum_temperature"] >= 17) & (df["maximum_temperature"] <= 25)].count())
    break

88
